In [0]:
from pyspark.sql.functions import current_timestamp

# Load the source data
player_activity_df = spark.table("dev_catalog.silver.player_activity")

# Select the required field and add insert_date and update_date
dim_play_mode_df = player_activity_df.select("PlayMode") \
                                     .distinct() \
                                     .withColumn("insert_date", current_timestamp()) \
                                     .withColumn("update_date", current_timestamp())

# Merge the dimension table into the gold catalog
dim_play_mode_df.createOrReplaceTempView("dim_play_mode_temp")

query = ("""
MERGE INTO dev_catalog.gold.dim_play_mode AS target
USING dim_play_mode_temp AS source
ON target.PlayMode = source.PlayMode
WHEN MATCHED THEN
  UPDATE SET target.update_date = source.update_date
WHEN NOT MATCHED THEN
  INSERT (PlayMode, insert_date, update_date)
  VALUES (source.PlayMode, source.insert_date, source.update_date)
""")

spark.sql(query)

# Display the dimension table
#display(spark.table("dev_catalog.gold.dim_play_mode"))